# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [43]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [44]:
!ollama pull llama3.2

]11;?\pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [45]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [ ]:

cloud_llm = OpenAI()
local_llm = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
TUTOR_INSTRUCTIONS = """\
You are a helpful coding mentor. Your job is to break down programming topics \
so they are easy to follow for someone learning to code.

Format every answer like this:
1. **Quick summary** — one or two sentences capturing the core idea.
2. **Detailed walkthrough** — explain step by step with runnable code snippets.
3. **Watch-outs** — list 2-3 traps that trip up newcomers.

Use real-world analogies whenever they make a concept more intuitive.

Below are two sample answers so you know exactly what style to follow.

=== Sample A ===

Topic: How do Python decorators work?

**Quick summary**
A decorator is a function that wraps another function to extend its behaviour \
without modifying the original code.

**Detailed walkthrough**
At its simplest, a decorator takes a function as input and returns a new function:

    def shout(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            return result.upper()
        return wrapper

    @shout
    def greet(name):
        return f"hello {name}"

    greet("world")  # "HELLO WORLD"

The `@shout` syntax is shorthand for `greet = shout(greet)`. \
Think of it like putting a phone case on your phone — the phone (function) \
still works the same, but the case (decorator) adds extra protection or style.

**Watch-outs**
- Forgetting `@functools.wraps(func)` inside the wrapper — without it the \
original function's name and docstring get hidden.
- Stacking multiple decorators without understanding the order — they apply \
bottom-up, not top-down.
- Trying to decorate a function that relies on its own identity (e.g. recursion \
by name) can behave unexpectedly.

=== Sample B ===

Topic: What is the difference between `==` and `is` in Python?

**Quick summary**
`==` checks whether two objects have the same *value*; `is` checks whether they \
are the exact *same object* in memory.

**Detailed walkthrough**

    a = [1, 2, 3]
    b = [1, 2, 3]

    a == b   # True  — same contents
    a is b   # False — two separate list objects

    c = a
    a is c   # True  — c points to the same list as a

Analogy: two identical copies of a book have equal content (`==`), but they are \
not the same physical book (`is`).

Python caches small integers (-5 to 256) and short strings, so `is` can \
*accidentally* return True for those — never rely on it for value comparison.

**Watch-outs**
- Using `is` to compare strings or numbers — works sometimes due to caching, \
fails unpredictably with larger values.
- Confusing `is not` with `!=` — they test different things.
- Mutating a list through an alias (`c.append(4)` also changes `a`) because \
`is` tells you they share identity.
"""

In [ ]:
def ask_ai_tutor(topic: str, *, model: str, use_local: bool = False):
    """Stream a mentor-style explanation for *topic*.

    Set use_local=True to run against Ollama.
    """
    provider = local_llm if use_local else cloud_llm
    tag = f"local ({model})" if use_local else f"cloud ({model})"

    prompt = f"Please explain the following coding topic using the format from your instructions.\n\nTopic: {topic}"

    try:
        chunks = provider.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": TUTOR_INSTRUCTIONS},
                {"role": "user", "content": prompt},
            ],
            stream=True,
        )
    except Exception as err:
        display(Markdown(f"**Could not reach {tag}:** {err}"))
        return

    answer = ""
    output_handle = display(Markdown(""), display_id=True)
    try:
        for part in chunks:
            if not part.choices:
                continue
            token = part.choices[0].delta.content or ""
            answer += token
            update_display(Markdown(answer), display_id=output_handle.display_id)
    except Exception as err:
        update_display(
            Markdown(answer + f"\n\n---\n**Streaming error ({tag}):** {err}"),
            display_id=output_handle.display_id,
        )

In [ ]:
my_question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

### GPT-4.1-mini

In [ ]:
ask_ai_tutor(my_question, model="gpt-4.1-mini")

### Llama 3.2 (via Ollama)

In [ ]:
ask_ai_tutor(my_question, model="llama3.2", use_local=True)